In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [6]:
# 데이터 불러오기 및 전처리
df=pd.read_csv('cow.csv')
df.rename(columns={'경락일자':'date'},inplace=True)
df.rename(columns={'1':'price'},inplace=True)
df['datetime'] = df['date'].apply(lambda x: pd.to_datetime(f"20{x}", format='%Y-%b'))
df['datetime']=df['datetime'].dt.strftime('%Y-%m')
df

,date,price,datetime
0,13-Jan,14498.31766,2013-01
1,13-Feb,13396.67663,2013-02
2,13-Mar,12914.19227,2013-03
3,13-Apr,12780.23766,2013-04
4,13-May,12340.69142,2013-05
...,...,...,...
127,23-Aug,14315.10509,2023-08
128,23-Sep,15991.13868,2023-09
129,23-Oct,14802.68385,2023-10
130,23-Nov,14399.18353,2023-11


In [13]:
model = SARIMAX(df['price'], 
                order=(1, 0, 0),    # ARIMA의 (p, d, q)
                seasonal_order=(1, 0, 0, 12))  # 계절적 (P, D, Q, m)
model_fit = model.fit(disp=False)
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                              price   No. Observations:                  132
Model:             SARIMAX(1, 0, 0)x(1, 0, 0, 12)   Log Likelihood               -1058.025
Date:                            Wed, 21 Aug 2024   AIC                           2122.050
Time:                                    22:39:35   BIC                           2130.698
Sample:                                         0   HQIC                          2125.564
                                            - 132                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9972      0.006    178.040      0.000       0.986       1.008
ar.S.L12       0.3392      0.082   

In [8]:
# 미래 예측 수행 (12개월 예측)
forecast = model_fit.get_forecast(steps=12)
forecast_values = forecast.predicted_mean
conf_int = forecast.conf_int()

In [11]:
forecast_df = pd.DataFrame({
    'Forecast': forecast_values,
    'Conf_int_low': conf_int.iloc[:, 0],
    'Conf_int_high': conf_int.iloc[:, 1]
})

In [12]:
print(forecast_df)


         Forecast  Conf_int_low  Conf_int_high
132  12797.141817  11401.580238   14192.703397
133  13070.488342  11099.588248   15041.388436
134  12954.127630  10543.603960   15364.651301
135  12616.369570   9836.768422   15395.970719
136  12173.760627   9070.346336   15277.174919
137  12419.604705   9024.656143   15814.553267
138  12519.851153   8857.921761   16181.780544
139  12948.332862   9038.935052   16857.730672
140  13494.521424   9353.660873   17635.381975
141  13069.110182   8710.235509   17427.984855
142  12910.024396   8344.650939   17475.397853
143  12416.594841   7654.733694   17178.455988


In [ ]:
#신뢰구간이 자꾸 넓어짐